In [1]:
import psycopg2
import itertools
import datetime

In [2]:
host = 'localhost'
port = 5432
database = 'msds691'
user = 'postgres'
password = ''

prev_min_range = 10
foll_min_range = 10

negative_case_interval = 240
negative_case_after_meal = 60
negative_case_before_meal = 60

In [3]:
def pretty_print(array):
    for item in array:
        print(','.join(str(i) for i in item))


In [4]:
conn = psycopg2.connect(host=host, port=port,
                        database=database, user=user, password=password)
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ)
cur = conn.cursor()

## create postgres tables

In [5]:
command = f"""
DROP TABLE IF EXISTS cgms CASCADE;
CREATE TABLE cgms(
        id VARCHAR(30),
        time_value TIMESTAMP,
        Flow DOUBLE PRECISION,
        CO2_Outlet DOUBLE PRECISION,
        O2_Outlet DOUBLE PRECISION,
        CO2_Inlet DOUBLE PRECISION,
        O2_Inlet DOUBLE PRECISION,
        VCO2 DOUBLE PRECISION,
        VO2 DOUBLE PRECISION,
        RQ DOUBLE PRECISION,
        EE DOUBLE PRECISION,
        Heart_Rate DOUBLE PRECISION,
        Respiratory_Rate DOUBLE PRECISION,
        T_Core DOUBLE PRECISION,
        T_Skin DOUBLE PRECISION,
        Glucose DOUBLE PRECISION,
        PRIMARY KEY (id, time_value)
);
COPY cgms FROM '/Users/dwoodbridge/Research/CGMS/cgms.csv' CSV HEADER;

DROP TABLE IF EXISTS meal_time CASCADE;
CREATE TABLE meal_time
(
        id VARCHAR(30),
        pattern VARCHAR(20),
        time_value TIMESTAMP,
        row INTEGER,
        PRIMARY KEY (id, time_value)
);
COPY meal_time FROM '/Users/dwoodbridge/Research/CGMS/meal_time.csv' CSV HEADER;

DROP VIEW IF EXISTS cgms_meal_time_view;
CREATE VIEW cgms_meal_time_view AS
SELECT cgms.id, cgms.time_value, Flow, CO2_Outlet, O2_Outlet, CO2_Inlet, O2_Inlet, VCO2, VO2,
RQ, EE, Heart_Rate, Respiratory_Rate, T_Core, T_Skin, Glucose, CASE WHEN pattern IS NOT NULL THEN True END AS meal
FROM cgms
LEFT JOIN meal_time
ON cgms.id = meal_time.id AND cgms.time_value = meal_time.time_value
ORDER BY id, time_value;
"""
cur.execute(command)
conn.commit()

## Distinct IDs

In [5]:
command = """SELECT DISTINCT(id) FROM meal_time;"""
cur.execute(command)
conn.commit()
data = cur.fetchall()

In [43]:
id_list = list()
for i in range(0, len(data)):
    id_list.append(data[i][0])

In [46]:
data_size = foll_min_range + prev_min_range - 1  

## Create positive cases

In [47]:
ml_positive_input = list()

for id in id_list:
    command = f"""SELECT id, time_value FROM meal_time WHERE id = '{id}';"""
    cur.execute(command)
    conn.commit()
    id_times = cur.fetchall()

    for id_time in id_times:
        id = id_time[0]
        meal_time = id_time[1]

        command = f"""SELECT * FROM cgms_meal_time_view WHERE id = '{id}' AND\
                      time_value >= TIMESTAMP '{meal_time}' - INTERVAL '{prev_min_range} minutes' AND\
                      time_value <= TIMESTAMP '{meal_time}' + INTERVAL '{foll_min_range} minutes';"""
        cur.execute(command)
        conn.commit()
        readings = cur.fetchall()

        input = list()
        if(len(readings) == data_size):
            for i in range(0, data_size):
                input.append(readings[i][2:-1])
            input.append([1]) # label
            ml_positive_input.append(list(itertools.chain(*input)))
        
        else: # Leave Missing Data as None
            command = f"""SELECT MIN(time_value), MAX(time_value)
                          FROM cgms_meal_time_view WHERE id = '{id}' AND\
                          time_value >= TIMESTAMP '{meal_time}' - INTERVAL '{prev_min_range} minutes' AND\
                          time_value <= TIMESTAMP '{meal_time}' + INTERVAL '{foll_min_range} minutes';"""
            cur.execute(command)
            conn.commit()
            reading = cur.fetchone()
            first, last = reading[0], reading[1]

            min_time = meal_time - datetime.timedelta(minutes=prev_min_range)
            first = first - datetime.timedelta(minutes=1)
            while(first > min_time):
                prev_null_reading = [(first, None, None, None, None, None, None, None,\
                                     None, None, None, None, None, None,\
                                     None, None, None)]
                readings[:0] = prev_null_reading
                
                first = first - datetime.timedelta(minutes=1)
            
            max_time = meal_time + datetime.timedelta(minutes=foll_min_range)
            last = last + datetime.timedelta(minutes=1)
            while(last < max_time):
                foll_null_reading = (last, None, None, None, None, None, None, None,\
                                     None, None, None, None, None, None,\
                                     None, None, None)
                readings.append(foll_null_reading)
                
                last = last + datetime.timedelta(minutes=1)
            
            input = list()
            if(len(readings) == data_size):
                for i in range(0, data_size):
                    input.append(readings[i][2:-1])
                input.append([1]) # label
                ml_positive_input.append(list(itertools.chain(*input)))

## Create negative cases

In [48]:
command =  f"""
            SELECT id, time_value, next_time_value
            FROM
            (
                SELECT id, time_value, 
                       LEAD(time_value) OVER (PARTITION BY id ORDER BY time_value) AS next_time_value
                FROM cgms_meal_time_view
                WHERE meal = True
            ) AS id_time_next_time
            WHERE next_time_value - time_value > INTERVAL '{negative_case_interval} minutes';
            """
cur.execute(command)
conn.commit()
data = cur.fetchall()

In [49]:
ml_negative_input = list()
for dat in data:
    neg_eating_start = dat[1] + datetime.timedelta(minutes=negative_case_after_meal)
    
    neg_eating_end = dat[2] - datetime.timedelta(minutes=negative_case_before_meal)
    id = dat[0]
    
    command = f"""SELECT * 
               FROM cgms_meal_time_view 
               WHERE id = '{id}'
               AND time_value >= '{neg_eating_start}'
               AND time_value <= '{neg_eating_end}';"""
    cur.execute(command)
    conn.commit()
    neg_data = cur.fetchall()
    
    for i in range(0, len(neg_data) -  (data_size + 1)):
        length = 0
        readings = list()
        while (length < data_size):
            readings.append(neg_data[i + length][2:-1])
            length = length + 1
        readings.append([0])
        ml_negative_input.append(list(itertools.chain(*readings)))


In [51]:
import csv

file_name = '{}_{}_{}_{}_{}.csv'.format(
    'cgms', prev_min_range, foll_min_range, negative_case_after_meal, negative_case_before_meal)
print(file_name)
file = open(file_name, 'w', newline='')

# writing the data into the file
with file:
    write = csv.writer(file)
    write.writerows(ml_positive_input)
    write.writerows(ml_negative_input)
    file.close()

cgms_10_10_60_60.csv


In [52]:
conn.close()